# TODO

## 직장인->거주자 로 바꾸는 chain을 적용해보자

1. 문서의 내용을 읽는다.
2. 문서를 쪼갠다.
 - 문서가 길면 토큰수 초과로 답변을 생성하지 못할 수 있다.
 - 문서가 길면 (인풋이 길면) 답변 생성에 시간이 너무 많이 소요된다.
 3. 임베딩 -> 벡터 데이터베이스에 저장
 4. 질문이 있을 때, 벡터 데이터베이스에 유사도 검색
 5. 유사도 검색으로 가져온 문서를 LLM에 질문과 같이 전달

In [12]:
#%pip install --upgrade --quiet  docx2txt langchain-community

## Pinecone VectorStore를 이용

In [1]:
%pip install -qU langchain langchain-pinecone langchain-openai

Note: you may need to restart the kernel to use updated packages.


## Load and Split

In [2]:
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import CharacterTextSplitter

# text splitter는 보통 두개의 파라미터를 가짐. 1. 청크사이즈, 2. 청크오버랩
text_splitter = CharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=200,
)

loader = Docx2txtLoader("tax-markdown.docx")
# document = loader.load()
# document를 loader.load()대신 loader.load_and_split(text_splitter)으로 불러오면 청크사이즈와 청크오버랩을 고려해서 쪼개진 문서를 반환함.
document_list = loader.load_and_split(text_splitter=text_splitter)

In [3]:
len(document_list)

182

In [ ]:
# %pip install -qU langchain_text_splitters

Note: you may need to restart the kernel to use updated packages.


## Embedding

In [4]:
from dotenv import load_dotenv
from langchain_openai import OpenAIEmbeddings

load_dotenv()
embeddings = OpenAIEmbeddings(model = 'text-embedding-3-large')



이제 우리가 임베딩 한 결과를 VectorDB에 넣어주어야함.

### pineconeDB를 이용

In [5]:
import getpass
import os

from pinecone import Pinecone
from langchain_pinecone import PineconeVectorStore

if not os.getenv("PINECONE_API_KEY"):
    os.environ["PINECONE_API_KEY"] = getpass.getpass("Enter your Pinecone API key: ")

pinecone_api_key = os.environ.get("PINECONE_API_KEY")

pc = Pinecone(api_key=pinecone_api_key)

index_name = 'tax-markdown-index'
index = pc.Index(index_name)
# database = PineconeVectorStore.from_documents(document_list, embeddings, index_name=index_name) #이거 공식문서에서 이제 빠짐.
database = PineconeVectorStore( index=index, embedding=embeddings)



/opt/miniconda3/envs/langchain_rag-env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/opt/miniconda3/envs/langchain_rag-env/lib/python3.11/site-packages/langchain_pinecone/__init__.py:3: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  from langchain_pinecone.vectorstores import Pinecone, PineconeVectorStore


In [6]:
# 이제 add_documents함수를 이용해 문서를 추가할 수 있음.
database.add_documents(documents=document_list)

['d3696aac-6ee1-4362-a227-7d0385bb88c6',
 'c60dca7d-dc5f-4329-8dc8-6c8ce3130ff1',
 '9fe3059e-40d5-4fd5-b070-f53196e061a0',
 '8e68c332-cd37-4990-9a26-8f22c0a81189',
 '6d19ea65-9aa0-44bd-8bbb-99800581db5c',
 'b0b07503-f117-4817-80be-94af335da4f7',
 'c9ff4b9c-781f-40cb-b693-030b38d5829c',
 'b09aeb96-e8a8-4f64-9746-39f73e65fa51',
 '2bd4b047-8f93-440d-91e3-2acbf69162c1',
 '8a0ca6a7-478b-4df4-9eb9-d8fe6c91f110',
 '7bbe9d8b-ecb9-4759-8ae8-ef5f6de9a58c',
 '7a05819d-7d70-4453-b085-fea223434d95',
 '431ed341-d54a-4e91-b13f-1b25952280a3',
 'ee836602-1623-473d-b126-d24787099512',
 '106bc014-68e0-4705-9c7a-15611f7a5df3',
 'e34a3f96-dfa2-43e4-9ea6-1ab7a23d72ff',
 'bf515ba0-f622-4e71-88ca-35e3b790c29f',
 '5d161734-e3ae-481f-8f31-0e28b81b4162',
 '38a176f5-3ebc-4e7f-83d1-c2b48d2399d2',
 'bf05fa7f-c031-4bf4-83e5-ecf8b77a05c6',
 'd2681ede-5ff0-43fc-8323-6355385ab27c',
 'e1309588-b5d4-4cc9-8fba-83b5677f872a',
 'cf1aaaa7-0962-448b-8f1f-5f757c923c64',
 '52aed6eb-6395-4631-a7fb-9eb798406ca0',
 'aeff039e-bf92-

질문던지기

In [7]:
query = "연봉 5000만원의 직장인은 소득세가 얼마인가요?"
# 이건 실무 잡이야기이긴 한데, 존댓말로 물어보면 답변을 더 잘해준다고 합니다. 영어질문은 Please 붙여서 하면 더 답변이 잘나온다는 이야기가 있습니다.
retrieved_docs = database.similarity_search(query, k=5) # similarity_search의 디폴트 값이 4이므로 k값 설정을 안해두면 4개의 문서를 반환할 것임.

In [8]:
document_list[52]

Document(metadata={'source': 'tax-markdown.docx', 'text': '가. 종교단체에 기부한 금액이 있는 경우\n\n\u3000\u3000한도액 = [종합소득금액(제62조에 따른 원천징수세율을 적용받는 이자소득 및 배당소득은 제외한다)에서 제1호에 따른 기부금을 뺀 금액을 말하며, 이하 이 항에서 “소득금액”이라 한다] × 100분의 10 + [소득금액의 100분의 20과 종교단체 외에 기부한 금액 중 적은 금액]\n\n나. 가목 외의 경우\n\n\u3000\u3000한도액 = 소득금액의 100분의 30\n\n⑤ 제1항부터 제3항까지의 규정을 적용할 때 과세기간 종료일 이전에 혼인ㆍ이혼ㆍ별거ㆍ취업 등의 사유로 기본공제대상자에 해당되지 아니하게 되는 종전의 배우자ㆍ부양가족ㆍ장애인 또는 과세기간 종료일 현재 65세 이상인 사람을 위하여 이미 지급한 금액이 있는 경우에는 그 사유가 발생한 날까지 지급한 금액에 제1항부터 제3항까지의 규정에 따른 율을 적용한 금액을 해당 과세기간의 종합소득산출세액에서 공제한다.\n\n⑥ 제1항부터 제4항까지의 규정에 따른 공제는 해당 거주자가 대통령령으로 정하는 바에 따라 신청한 경우에 적용한다.\n\n⑦ 국세청장은 제3항제2호라목에 따른 교육비가 세액공제 대상에 해당하는지 여부를 확인하기 위하여 「한국장학재단 설립 등에 관한 법률」 제6조에 따른 한국장학재단 등 학자금 대출ㆍ상환업무를 수행하는 대통령령으로 정하는 기관(이하 이 항에서 “한국장학재단등”이라 한다)에 학자금대출 및 원리금 상환내역 등 대통령령으로 정하는 자료의 제공을 요청할 수 있다. 이 경우 요청을 받은 한국장학재단등은 특별한 사유가 없으면 그 요청에 따라야 한다.<신설 2016. 12. 20.>\n\n⑧ 제4항에도 불구하고 2024년 1월 1일부터 2024년 12월 31일까지 지급한 기부금을 해당 과세기간의 합산과세되는 종합소득산출세액(필요경비에 산입한 기부금이 있는 경우 사업소득에 대한 산출세액은 제외한다)에서 공제하는 경우에는

In [9]:
retrieved_docs

[Document(id='2378e94f-cc58-467a-be98-999730428de4', metadata={'source': 'tax-markdown.docx'}, page_content='[전문개정 2009. 12. 31.]\n\n제10조(납세지의 변경신고) 거주자나 비거주자는 제6조부터 제9조까지의 규정에 따른 납세지가 변경된 경우 변경된 날부터 15일 이내에 대통령령으로 정하는 바에 따라 그 변경 후의 납세지 관할 세무서장에게 신고하여야 한다.\n\n[전문개정 2009. 12. 31.]\n\n제11조(과세 관할) 소득세는 제6조부터 제10조까지의 규정에 따른 납세지를 관할하는 세무서장 또는 지방국세청장이 과세한다.\n\n[전문개정 2009. 12. 31.]\n\n제2장 거주자의 종합소득 및 퇴직소득에 대한 납세의무 <개정 2009. 12. 31.>\n\n제1절 비과세 <개정 2009. 12. 31.>\n\n제12조(비과세소득) 다음 각 호의 소득에 대해서는 소득세를 과세하지 아니한다. <개정 2010. 12. 27., 2011. 7. 25., 2011. 9. 15., 2012. 2. 1., 2013. 1. 1., 2013. 3. 22., 2014. 1. 1., 2014. 3. 18., 2014. 12. 23., 2015. 12. 15., 2016. 12. 20., 2018. 3. 20., 2018. 12. 31., 2019. 12. 10., 2019. 12. 31., 2020. 6. 9., 2020. 12. 29., 2022. 8. 12., 2022. 12. 31., 2023. 8. 8., 2023. 12. 31., 2024. 12. 31.>\n\n1. 「공익신탁법」에 따른 공익신탁의 이익\n\n2. 사업소득 중 다음 각 목의 어느 하나에 해당하는 소득\n\n가. 논ㆍ밭을 작물 생산에 이용하게 함으로써 발생하는 소득\n\n나. 1개의 주택을 소유하는 자의 주택임대소득(제99조에 따른 기준시가가 12억원을 초과하는 주택 및 국외에 소재하는 주택의 임대

In [10]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model = 'gpt-4o-mini', temperature = 0)

In [11]:
prompt = f"""[Identity]
- 당신은 최고의 한국 소득세 전문가입니다.
- [context]를 참고해서 사용자의 질문에 답변해주세요.

[context]
{retrieved_docs}

Question: {query}

"""

In [12]:
ai_message = llm.invoke(prompt)

In [13]:
ai_message.content

'연봉 5,000만원의 직장인이 납부해야 할 소득세를 계산하기 위해서는 소득세율을 적용해야 합니다. 아래는 5,000만원에 해당하는 소득세 계산 과정입니다.\n\n1. **종합소득 과세표준**: 5,000만원\n2. **세율 적용**:\n   - 1,400만원 이하: 1,400만원 × 6% = 84만원\n   - 1,400만원 초과 ~ 5,000만원 이하: (5,000만원 - 1,400만원) × 15% = 3,600만원 × 15% = 540만원\n\n3. **총 소득세**:\n   - 84만원 + 540만원 = 624만원\n\n따라서, 연봉 5,000만원의 직장인은 약 624만원의 소득세를 납부해야 합니다.'

'연봉 5,000만원의 직장인이 납부해야 할 소득세를 계산해보겠습니다. 소득세는 종합소득 과세표준에 따라 세율이 적용됩니다.\n\n1. **연봉 5,000만원의 경우**:\n   - 5,000만원은 **1,400만원 초과 ~ 5,000만원 이하** 구간에 해당합니다.\n   - 이 구간의 세율은 다음과 같습니다:\n     - 1,400만원 이하: 과세표준 × 6%\n     - 1,400만원 초과 ~ 5,000만원 이하: 84만원 + (1,400만원 초과분 × 15%)\n\n2. **계산**:\n   - 5,000만원에서 1,400만원을 빼면 3,600만원이 남습니다.\n   - 따라서, 세액은 다음과 같이 계산됩니다:\n     - 기본 세액: 84만원\n     - 추가 세액: 3,600만원 × 15% = 540만원\n\n3. **총 소득세**:\n   - 총 소득세 = 기본 세액 + 추가 세액\n   - 총 소득세 = 84만원 + 540만원 = 624만원\n\n따라서, 연봉 5,000만원의 직장인은 약 **624만원**의 소득세를 납부해야 합니다.'

## RetrievalQA Chain

In [43]:
%pip install -U langchain langchainhub --quiet

Note: you may need to restart the kernel to use updated packages.


In [15]:
from langchain import hub

prompt = hub.pull("rlm/rag-prompt")

In [45]:
prompt

ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"), additional_kwargs={})])

In [16]:
## RetrievalQA Chain을 어떻게 사용하는지 알아보자.
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever = database.as_retriever(),
    chain_type_kwargs = {"prompt":prompt}
    
)

# query -> 직장인 -> 거주자로 바꾸는 chain을 추가
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

dictionary = ["사람을 나타내는 표현 -> 거주자"]

prompt = ChatPromptTemplate.from_template(f"""
                사용자의 질문을 보고, 우리의 사전을 참고해서 사용자의 질문을 변경해주세요.
                만약 변경할 필요가 없다고 판단된다면, 사용자의 질문을 변경하지 않아도 됩니다.
                사전 : {dictionary}
                질문 : {{question}}
                """)

dictionary_chain = prompt | llm | StrOutputParser() 

In [17]:
tax_chain = {"query":dictionary_chain} | qa_chain

In [18]:
ai_response = tax_chain.invoke({"question":query})

In [19]:
ai_response

{'query': '연봉 5000만원의 거주자는 소득세가 얼마인가요?',
 'result': '연봉 5000만원의 거주자는 소득세로 84만원 + (1,400만원 초과분 × 15%)를 납부해야 합니다. 즉, 84만원 + (3,600만원 × 15%) = 84만원 + 540만원 = 624만원이 됩니다. 따라서 총 소득세는 624만원입니다.'}

In [20]:
ai_message = qa_chain({"query":query})

/var/folders/7z/w98360257114dm09jq3nwdq40000gn/T/ipykernel_27957/980961933.py:1: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  ai_message = qa_chain({"query":query})


In [21]:
ai_message

{'query': '연봉 5000만원의 직장인은 소득세가 얼마인가요?',
 'result': '연봉 5000만원의 직장인은 소득세로 약 84만원 + (5000만원 초과분 × 15%)를 납부해야 합니다. 5000만원을 초과하지 않으므로, 소득세는 84만원입니다. 따라서, 연봉 5000만원의 직장인은 소득세로 약 84만원을 납부하게 됩니다.'}